In [1]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip

--2025-12-02 14:24:10--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-02T15%3A18%3A00Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-02T14%3A17%3A03Z&ske=2025-12-02T15%3A18%3A00Z&sks=b&skv=2018-11-09&sig=%2BNMYcMwnvZkPV5ks47IPEnTXLfZ3KXcRdDGLLI3%2BZPE%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDY4NzI1MCwibmJmIjoxNzY0Njg1NDUwLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi

In [2]:
!unzip data.zip

Archive:  data.zip
replace data/test/curly/03312ac556a7d003f7570657f80392c34.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [3]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torchsummary import summary
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torch.utils.data import random_split

In [5]:
print(torch.__version__)

2.9.0+cu126


In [6]:
class MyModel(nn.Module):
  def __init__(self):
    super(MyModel, self).__init__()
    # 3 input image channels (RGB), 32 output channels, 3x3 square convolution kernel
    self.conv1 = nn.Conv2d(3, 32, 3)
    self.relu = nn.ReLU()
    self.pool = nn.MaxPool2d(2, 2)
    self.vectorize = nn.Flatten()
    self.fc1 = nn.Linear(32 * 99 * 99, 64)
    self.relu2 = nn.ReLU()
    self.output = nn.Linear(64, 1)

  def forward(self, x):
      x = self.relu(self.conv1(x))      # Conv + ReLU
      x = self.pool(x)                  # Max pooling
      x = self.vectorize(x)             # Flatten to 1D
      x = self.relu2(self.fc1(x))       # Linear + ReLU
      x = self.output(x)                # Final output (raw logits)
      return x


In [7]:
model = MyModel()
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
criterion = nn.BCEWithLogitsLoss()

In [8]:
print(model)

MyModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (vectorize): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
  (relu2): ReLU()
  (output): Linear(in_features=64, out_features=1, bias=True)
)


In [9]:
# Question 2 - Total parameters

total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 20073473


In [10]:
# Generators and Training

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [11]:
# Apply transforms to dataset
train_dataset = datasets.ImageFolder('/content/data/train', transform=train_transforms)

# Split train_dataset into train/validation
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_subset, validation_subset = random_split(train_dataset, [train_size, val_size])

# Create loaders
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(validation_subset, batch_size=20, shuffle=False)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [13]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_subset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6459, Acc: 0.6200, Val Loss: 0.6745, Val Acc: 0.5875
Epoch 2/10, Loss: 0.5688, Acc: 0.6887, Val Loss: 0.5347, Val Acc: 0.7562
Epoch 3/10, Loss: 0.5349, Acc: 0.6963, Val Loss: 0.4803, Val Acc: 0.7750
Epoch 4/10, Loss: 0.4737, Acc: 0.7688, Val Loss: 0.4059, Val Acc: 0.7937
Epoch 5/10, Loss: 0.4177, Acc: 0.7975, Val Loss: 0.3877, Val Acc: 0.8000
Epoch 6/10, Loss: 0.3725, Acc: 0.8275, Val Loss: 0.5141, Val Acc: 0.7375
Epoch 7/10, Loss: 0.3324, Acc: 0.8400, Val Loss: 0.2517, Val Acc: 0.9250
Epoch 8/10, Loss: 0.2679, Acc: 0.8862, Val Loss: 0.2308, Val Acc: 0.9187
Epoch 9/10, Loss: 0.1824, Acc: 0.9287, Val Loss: 0.1251, Val Acc: 0.9812
Epoch 10/10, Loss: 0.1484, Acc: 0.9425, Val Loss: 0.1209, Val Acc: 0.9563


In [14]:
# Question 3: Median of training accuracy for all the epochs
median = np.median(history['acc'])
print(f"Median is: {median:.2f}")

Median is: 0.81


In [15]:
# Question 4: What is the standard deviation of training loss for all the epochs for this model?
std_dev = np.std(history['loss'])
print(f"Standard deviation is: {std_dev:.3f}")

Standard deviation is: 0.157


In [16]:
# Data Augmentation

transforms.RandomRotation(50),
transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
transforms.RandomHorizontalFlip(),

(RandomHorizontalFlip(p=0.5),)

In [17]:
num_epochs = 20
augmented_history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(10, num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    augmented_history['loss'].append(epoch_loss)
    augmented_history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_subset)
    val_epoch_acc = correct_val / total_val
    augmented_history['val_loss'].append(val_epoch_loss)
    augmented_history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 11/20, Loss: 0.1039, Acc: 0.9625, Val Loss: 0.1870, Val Acc: 0.9125
Epoch 12/20, Loss: 0.1472, Acc: 0.9487, Val Loss: 0.1637, Val Acc: 0.9125
Epoch 13/20, Loss: 0.0670, Acc: 0.9825, Val Loss: 0.0473, Val Acc: 0.9938
Epoch 14/20, Loss: 0.0256, Acc: 0.9988, Val Loss: 0.0209, Val Acc: 1.0000
Epoch 15/20, Loss: 0.0145, Acc: 1.0000, Val Loss: 0.0132, Val Acc: 1.0000
Epoch 16/20, Loss: 0.0100, Acc: 1.0000, Val Loss: 0.0089, Val Acc: 1.0000
Epoch 17/20, Loss: 0.0076, Acc: 1.0000, Val Loss: 0.0070, Val Acc: 1.0000
Epoch 18/20, Loss: 0.0059, Acc: 1.0000, Val Loss: 0.0064, Val Acc: 1.0000
Epoch 19/20, Loss: 0.0051, Acc: 1.0000, Val Loss: 0.0050, Val Acc: 1.0000
Epoch 20/20, Loss: 0.0043, Acc: 1.0000, Val Loss: 0.0044, Val Acc: 1.0000


In [18]:
# Question 5: What is the mean of test loss for all the epochs for the model trained with augmentations?
mean = np.mean(augmented_history['loss'])
print(f"Mean is: {mean:.3f}")

Mean is: 0.039


In [19]:
# Question 6: What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?
mean = np.mean(augmented_history['acc'][5:])
print(f"Mean is: {mean:.3f}")

Mean is: 1.000
